In [1]:
# 本节学习vector 对 vector的求导
# 也就是多个输入，多个输出的时候，怎么求导
# 基本原则与链式法则一致，求导的结果被称为雅可比(Jacobian)矩阵

In [ ]:
# N个输入，M个输出，求导结果为M*N大小的矩阵
# 每个输出对每个输入单独求导

# 示例：线性函数 y=Ax+b
# J= ∂y / ∂x = A

In [4]:
import torch
print(torch.__version__)

2.2.1+cu118


In [14]:
import torch

# 定义输入向量 x (n=2 维度)，设置 requires_grad=True
x = torch.tensor([0.0, 0.0], requires_grad=True)

# 定义矩阵 A (n=2, m=2) 和向量 b (m=2)
A = torch.tensor([[2.0, 3.0], [4.0, 5.0]])
b = torch.tensor([6.0, 7.0])

# 定义函数 func，计算 y = A @ x + b
def func(x):
    return A @ x + b

# 计算 Jacobian 矩阵
jacobian = torch.autograd.functional.jacobian(func, x)

print("Jacobian Matrix:")
print(jacobian)

Jacobian Matrix:
tensor([[2., 3.],
        [4., 5.]])


In [24]:
# 特殊情况 多输入 单输出

# “多输入单输出”的一阶导数结果，称之为梯度向量
# “多输入单输出”的二阶导数结果，称之为Hessian矩阵，梯度向量的每个结果对每个变量求导

import torch

# 定义多个输入变量，设置 requires_grad=True
x1 = torch.tensor(1.0, requires_grad=True)
x2 = torch.tensor(2.0, requires_grad=True)
x3 = torch.tensor(3.0, requires_grad=True)

# 定义计算输出的函数
def func(x1, x2, x3):
    return x1**2 + x2**3 + x3**4

# 计算输出
y = func(x1, x2, x3)

# 计算一阶导数（梯度向量）
gradient = torch.autograd.grad(y, (x1, x2, x3), create_graph=True)

print("Gradient Vector (一阶导数):")
print(torch.tensor([g.item() for g in gradient]))

# 计算二阶导数（Hessian 矩阵）
def compute_hessian(gradient, inputs):
    hessian = []
    for g in gradient:
        hessian_row = torch.autograd.grad(g, inputs, retain_graph=True, allow_unused=True)
        hessian_row = [hr if hr is not None else torch.zeros_like(inp) for hr, inp in zip(hessian_row, inputs)]
        hessian.append(torch.stack(hessian_row))
    return torch.stack(hessian)

# 计算 Hessian 矩阵
inputs = (x1, x2, x3)
hessian = compute_hessian(gradient, inputs)

print("\nHessian Matrix (二阶导数):")
print(hessian)

Gradient Vector (一阶导数):
tensor([  2.,  12., 108.])

Hessian Matrix (二阶导数):
tensor([[  2.,   0.,   0.],
        [  0.,  12.,   0.],
        [  0.,   0., 108.]])


理论验证

对于函数 $ y = x_1^2 + x_2^3 + x_3^4 $，其梯度和 Hessian 矩阵为：

1. 梯度向量

$$
\text{Gradient} = \left[ 2x_1, 3x_2^2, 4x_3^3 \right]
$$

代入 $ x_1 = 1.0 $, $ x_2 = 2.0 $, $ x_3 = 3.0 $，得到：

$$
\text{Gradient} = \left[ 2, 12, 108 \right]
$$

2. Hessian 矩阵

$$
\text{Hessian} = \begin{bmatrix}
2 & 0 & 0 \\
0 & 6x_2 & 0 \\
0 & 0 & 12x_3^2
\end{bmatrix}
$$

代入 $ x_1 = 1.0 $, $ x_2 = 2.0 $, $ x_3 = 3.0 $，得到：

$$
\text{Hessian} = \begin{bmatrix}
2 & 0 & 0 \\
0 & 12 & 0 \\
0 & 0 & 108
\end{bmatrix}
$$

In [27]:
# 多输入多输出求导的链式法则
# 与普通的链式法则类似 ∂y / ∂x = (∂y / ∂u) * (∂u / ∂x)

import torch

# 定义输入变量
x1 = torch.tensor(1.0, requires_grad=True)
x2 = torch.tensor(2.0, requires_grad=True)

# 定义中间变量
u1 = x1 + x2
u2 = x1 * x2

# 定义输出变量
y1 = u1**2
y2 = u2**2

# 计算输出对输入的导数
y = torch.stack([y1, y2])  # y 的形状是 [2]

# 设置 grad_outputs 为与 y 形状匹配的张量
grad_outputs = torch.eye(2)  # 形状是 [2, 2]

# 计算 Jacobian 矩阵
jacobian = []
for i in range(2):
    grad = torch.autograd.grad(y[i], [x1, x2], grad_outputs=torch.ones_like(y[i]), retain_graph=True)
    jacobian.append(grad)

# 将结果堆叠为 Jacobian 矩阵
jacobian = torch.stack([torch.stack(row) for row in jacobian])

# 打印结果
print("Jacobian Matrix:")
print(jacobian)

Jacobian Matrix:
tensor([[6., 6.],
        [8., 4.]])


In [28]:
# 特殊情况：单输入-多输出-单输出
# 大小为 1*2 的矩阵，乘以大小为 2*1 的矩阵，得到大小为 1*1 的矩阵
import torch

# 定义输入变量
x = torch.tensor(2.0, requires_grad=True)

# 定义中间变量
u1 = x**2
u2 = x**3

# 定义输出变量
y = u1 + u2

# 计算 y 对 x 的导数
y.backward()

# 打印结果
print("dy/dx:", x.grad)

dy/dx: tensor(16.)
